# Data Exploration and Analysis
The following notebook will take a look at the data sets that will be used in the recommendation engine and prep the data for use in the engine itself.

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Read in Data

In [2]:
animes_df = pd.read_csv('data/animes.csv')
users_df = pd.read_csv('data/profiles.csv')
reviews_df = pd.read_csv('data/reviews.csv')
animes_clean_df = pd.read_csv('data/animes-clean.csv')

print("Animes DF: {}\nUsers DF: {}\nReviews DF: {}".format(animes_df.shape, users_df.shape, reviews_df.shape))

Animes DF: (19311, 12)
Users DF: (81727, 5)
Reviews DF: (192112, 7)


In [3]:
show = animes_df[animes_df['title'] == 'Monster']
show = show.iloc[0]
show

uid                                                          19
title                                                   Monster
synopsis      Dr. Kenzou Tenma, an elite neurosurgeon recent...
genre         ['Drama', 'Horror', 'Mystery', 'Police', 'Psyc...
aired                               Apr 7, 2004 to Sep 28, 2005
episodes                                                   74.0
members                                                  459710
popularity                                                  159
ranked                                                     46.0
score                                                      8.69
img_url       https://cdn.myanimelist.net/images/anime/10/18...
link                   https://myanimelist.net/anime/19/Monster
Name: 744, dtype: object

### Animes DataFrame Exploration and Cleaning

In [4]:
animes_df.head(5)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


First I want to check for duplicate or null rows. If there are, we want to get rid of them. I purposly used this dataset because there were no null values. So I expect isnull().sum() to return 0.

In [5]:
animes_df.duplicated().sum()

2943

In [6]:
animes_df.drop_duplicates(inplace=True)

In [7]:
animes_df.duplicated().sum()

0

In [8]:
animes_df.genre.isnull().sum()

0

Perfect! One thing I noticed about the animes dataframe was that the genre for each anime seems to be a list of genres. I want to check what data type the genre column is and create a column for each genre type.

In [9]:
animes_df['genre'].dtype

dtype('O')

The data type of the genre column seems to be an Object, meaning a string. So I need to loop through the genre column and get a set of the genres.

In [10]:
genres = []

# for every anime, split the genre column value to get each genre type
for genre_set in animes_df.genre:
    values = genre_set.strip("[]").split(",")
    values = [w.strip()[1:-1] for w in values]
    
    # add genres to list
    genres.extend(values)

# drop all duplicate values
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))
print(genres)

The number of genres is 44.
{'', 'Game', 'Magic', 'Shoujo', 'Super Power', 'Shounen Ai', 'Dementia', 'Seinen', 'Hentai', 'Parody', 'Music', 'Samurai', 'Romance', 'Psychological', 'Comedy', 'Yaoi', 'Action', 'Demons', 'Shounen', 'Sci-Fi', 'Kids', 'Josei', 'Thriller', 'Slice of Life', 'Cars', 'Vampire', 'Historical', 'Shoujo Ai', 'Yuri', 'Drama', 'Supernatural', 'Mystery', 'Fantasy', 'Military', 'Mecha', 'Harem', 'Police', 'Horror', 'Martial Arts', 'Sports', 'Adventure', 'Ecchi', 'School', 'Space'}


Notice the first element is empty. This happend when calling set(genres). We can quickly delete that.

In [11]:
genres = list(genres)
genres.pop(0)
genres = sorted(genres)
print("The number of genres is {}.".format(len(genres)))
print(genres)

The number of genres is 43.
['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri']


Now that I have a list of all possible genre types. I wanted to make a column for each genre. The value of the column will be a 1 if an anime is listed under this genre or a 0 if it is not.

In [12]:
def split_genres(anime):
    '''
    Will split the genre column of any anime row and return a 1 if the anime is listed in that genre.
    
    INPUT:
    anime - a string of the genres column for a specific anime
    
    OUTPUT:
    1 - if anime is listed in genre
    0 - if anime is not listed in genre
    '''
    try:
        if anime.find(genre) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# create column for each genre and fill in columns
for genre in genres:
    animes_df[genre] = animes_df['genre'].apply(split_genres)

Now there is an easier way to identify the genres that an anime is listed as. This will be used for filtering and content based recommendations.

In [13]:
animes_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,...,0,0,0,1,0,0,0,0,0,0
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,...,0,0,0,0,0,0,0,0,0,0
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,...,0,0,0,0,0,0,0,0,0,0
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,...,0,0,0,0,0,0,0,0,0,0
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,...,0,0,0,0,0,1,0,1,0,0


## Create Decade columns for further filtering

In [14]:
def split_anime_decade(val, decade):
    try:
        year = val.split(',')[1]
        year = year.strip()[:4]
    except:
        year = val.strip()[:4]
    
    try:
        if decade == 'Pre 1970':
            if int(year) < 1970:
                return 1
            return 0
        if int(year) >= int(decade) and int(year) < int(decade) + 10:
            return 1
        else:
            return 0
    except:
        return 0

In [15]:
decades = ['Pre 1970s', '1970s', '1980s', '1990s', '2000s', '2010s']
for decade in decades:
    column = []
    for row in animes_df['aired']:
        column.append(split_anime_decade(row, decade[:-1]))
    animes_df[decade] = column
animes_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,...,Thriller,Vampire,Yaoi,Yuri,Pre 1970s,1970s,1980s,1990s,2000s,2010s
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,...,0,0,0,0,0,0,0,0,0,1
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,...,0,0,0,0,0,0,0,0,0,1
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,...,0,0,0,0,0,0,0,0,0,1
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,...,0,0,0,0,0,0,0,0,1,0
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,...,0,1,0,0,0,0,0,0,0,1


In [16]:
animes_df = animes_df.drop(['genre', 'aired'], axis=1)
animes_df.head()

,uid,title,synopsis,episodes,members,popularity,ranked,score,img_url,link,...,Thriller,Vampire,Yaoi,Yuri,Pre 1970s,1970s,1980s,1990s,2000s,2010s
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,...,0,0,0,0,0,0,0,0,0,1
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,...,0,0,0,0,0,0,0,0,0,1
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,...,0,0,0,0,0,0,0,0,0,1
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,...,0,0,0,0,0,0,0,0,1,0
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,...,0,1,0,0,0,0,0,0,0,1


In [17]:
animes_df.to_csv('./data/animes-clean.csv')

In [17]:
top_ranked = animes_df.sort_values(by='ranked', ascending=True)
top_ranked.head(20)

,uid,title,synopsis,episodes,members,popularity,ranked,score,img_url,link,...,Thriller,Vampire,Yaoi,Yuri,Pre 1970s,1970s,1980s,1990s,2000s,2010s
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,...,0,0,0,0,0,0,0,0,1,0
773,9253,Steins;Gate,The self-proclaimed mad scientist Rintarou Oka...,24.0,1331710,7,2.0,9.11,https://cdn.myanimelist.net/images/anime/5/731...,https://myanimelist.net/anime/9253/Steins_Gate,...,1,0,0,0,0,0,0,0,0,1
772,11061,Hunter x Hunter (2011),Hunter x Hunter is set in a world where Hunte...,148.0,1052761,20,3.0,9.11,https://cdn.myanimelist.net/images/anime/11/33...,https://myanimelist.net/anime/11061/Hunter_x_H...,...,0,0,0,0,0,0,0,0,0,1
771,32281,Kimi no Na wa.,"Mitsuha Miyamizu, a high school girl, yearns t...",1.0,1139878,15,4.0,9.09,https://cdn.myanimelist.net/images/anime/5/870...,https://myanimelist.net/anime/32281/Kimi_no_Na_wa,...,0,0,0,0,0,0,0,0,0,1
770,38524,Shingeki no Kyojin Season 3 Part 2,Seeking to restore humanity’s diminishing hope...,10.0,446370,175,5.0,9.07,https://cdn.myanimelist.net/images/anime/1517/...,https://myanimelist.net/anime/38524/Shingeki_n...,...,0,0,0,0,0,0,0,0,0,1
769,28977,Gintama°,"Gintoki, Shinpachi, and Kagura return as the f...",51.0,281594,351,6.0,9.05,https://cdn.myanimelist.net/images/anime/3/720...,https://myanimelist.net/anime/28977/Gintama%C2%B0,...,0,0,0,0,0,0,0,0,0,1
768,9969,Gintama',"After a one-year hiatus, Shinpachi Shimura ret...",51.0,278110,353,7.0,9.04,https://cdn.myanimelist.net/images/anime/4/503...,https://myanimelist.net/anime/9969/Gintama,...,0,0,0,0,0,0,0,0,0,1
767,820,Ginga Eiyuu Densetsu,The 150-year-long stalemate between the two in...,110.0,175423,620,8.0,9.03,https://cdn.myanimelist.net/images/anime/13/13...,https://myanimelist.net/anime/820/Ginga_Eiyuu_...,...,0,0,0,0,0,0,1,0,0,0
766,35180,3-gatsu no Lion 2nd Season,"Now in his second year of high school, Rei Kir...",22.0,169544,657,9.0,9.02,https://cdn.myanimelist.net/images/anime/3/884...,https://myanimelist.net/anime/35180/3-gatsu_no...,...,0,0,0,0,0,0,0,0,0,1
765,28851,Koe no Katachi,"As a wild youth, elementary school student Sho...",1.0,842277,53,10.0,9.01,https://cdn.myanimelist.net/images/anime/1122/...,https://myanimelist.net/anime/28851/Koe_no_Kat...,...,0,0,0,0,0,0,0,0,0,1


## Create Filter Functions

In [18]:
def get_top_rated(n, df=animes_df):
    '''
    INPUT:
    df - animes df from cells above
    n - number of recs to return
    
    OUTPUT:
    recs -  the name and img url of the all time top rated animes
    '''
    recs = []
    top_ranked = df.sort_values(by='ranked', ascending=True).drop_duplicates()
    
    for i in range(n):
        recs.append((top_ranked.iloc[i].title, top_ranked.iloc[i].img_url))
                    
    return recs

In [19]:
top_rated = get_top_rated(10)
print(top_rated[0])

('Fullmetal Alchemist: Brotherhood', 'https://cdn.myanimelist.net/images/anime/1223/96541.jpg')


In [20]:
print(top_rated[5])

('Gintama°', 'https://cdn.myanimelist.net/images/anime/3/72078.jpg')


In [21]:
def get_top_rated_genre(genre, n, df=animes_df):
    '''
    INPUT:
    genre - a string containing the genre that will be filtered by
    n - the number of recommendations to be returned
    df - the animes df from above
    
    OUTPUT:
    recs - a list of recommendations with title and url link
    '''
    
    recs = []
    genre_df = df[df[genre] == 1].sort_values(by='score', ascending=False).drop_duplicates()
    
    for i in range(n):
        recs.append((genre_df.iloc[i].title, genre_df.iloc[i].img_url))
    
    return recs

In [22]:
genre = 'Romance'
romance_recs = get_top_rated_genre(genre, 20)
romance_recs

[('Kimi no Na wa.', 'https://cdn.myanimelist.net/images/anime/5/87048.jpg'),
 ('Clannad: After Story',
  'https://cdn.myanimelist.net/images/anime/13/24647.jpg'),
 ('Shigatsu wa Kimi no Uso',
  'https://cdn.myanimelist.net/images/anime/3/67177.jpg'),
 ('Monogatari Series: Second Season',
  'https://cdn.myanimelist.net/images/anime/3/52133.jpg'),
 ('Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen',
  'https://cdn.myanimelist.net/images/anime/1807/102419.jpg'),
 ('Seishun Buta Yarou wa Yumemiru Shoujo no Yume wo Minai',
  'https://cdn.myanimelist.net/images/anime/1613/102179.jpg'),
 ('Howl no Ugoku Shiro',
  'https://cdn.myanimelist.net/images/anime/5/75810.jpg'),
 ('Suzumiya Haruhi no Shoushitsu',
  'https://cdn.myanimelist.net/images/anime/2/73842.jpg'),
 ('Yojouhan Shinwa Taikei',
  'https://cdn.myanimelist.net/images/anime/10/50457.jpg'),
 ('Bakuman. 3rd Season',
  'https://cdn.myanimelist.net/images/anime/6/41845.jpg'),
 ('Kara no Kyoukai 5: Mujun Rasen',
  'https://cdn.myanim

In [23]:
def get_top_rated_decade(decade, n, df=animes_df):
    '''
    INPUT:
    decade - a string containing the decade that will be filtered by
    n - the number of recommendations to be returned
    df - the animes df from above
    
    OUTPUT:
    recs - a list of recommendations with title and url link
    '''
    
    recs = []
    decade_df = df[df[decade] == 1].sort_values(by='score', ascending=False).drop_duplicates()
    
    for i in range(n):
        recs.append((decade_df.iloc[i].title, decade_df.iloc[i].img_url))
    
    return recs

In [24]:
decade = '1990s'
decades_df = get_top_rated_decade(decade, 20)
decades_df

[('Kaitei-koku no Koutsu Anzen',
  'https://cdn.myanimelist.net/images/anime/1957/99484.jpg'),
 ('Cowboy Bebop', 'https://cdn.myanimelist.net/images/anime/4/19644.jpg'),
 ('Mononoke Hime', 'https://cdn.myanimelist.net/images/anime/7/75919.jpg'),
 ('Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen',
  'https://cdn.myanimelist.net/images/anime/1807/102419.jpg'),
 ('Great Teacher Onizuka',
  'https://cdn.myanimelist.net/images/anime/13/11460.jpg'),
 ('Slam Dunk', 'https://cdn.myanimelist.net/images/anime/12/86890.jpg'),
 ('One Piece', 'https://cdn.myanimelist.net/images/anime/6/73245.jpg'),
 ('Neon Genesis Evangelion: The End of Evangelion',
  'https://cdn.myanimelist.net/images/anime/12/39305.jpg'),
 ('Time Slip 1923: Mori no Miracle Jishin Taiken',
  'https://cdn.myanimelist.net/images/anime/1511/98953.jpg'),
 ('Kenpuu Denki Berserk',
  'https://cdn.myanimelist.net/images/anime/12/18520.jpg'),
 ('Yuu☆Yuu☆Hakusho', 'https://cdn.myanimelist.net/images/anime/8/25152.jpg'),
 ('Hunter x

## Users Data

In [22]:
users_df.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [23]:
users_df.shape

(81727, 5)

In [24]:
reviews_df.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [26]:
user_item = reviews_df[['uid', 'anime_uid', 'score']]
user_item.head()

,uid,anime_uid,score
0,255938,34096,8
1,259117,34599,10
2,253664,28891,7
3,8254,2904,9
4,291149,4181,10


In [27]:
user_item_matrix = user_item.groupby(['uid', 'anime_uid'])['score'].max().unstack()

In [28]:
user_item_matrix.head()

anime_uid,1,5,6,7,8,15,16,17,18,19,...,40394,40420,40438,40477,40480,40489,40542,40693,40769,40807
uid,,,,,,,,,,,,,,,,,,,,,
1,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_movies_watched(user_id):
    

## Content Based Filtering

Finding similar tv shows based on similar genres

In [6]:
show_contents = animes_clean_df.iloc[:, 11:]
dot_prod_movies = show_contents.dot(np.transpose(show_contents))

In [60]:
print(type(dot_prod_movies))

<class 'pandas.core.frame.DataFrame'>


In [50]:
dot_prod_movies.head()

,0,1,2,3,4,5,6,7,8,9,...,16358,16359,16360,16361,16362,16363,16364,16365,16366,16367
0,6,4,2,3,1,2,1,2,2,1,...,4,2,1,3,2,3,2,3,1,4
1,4,6,2,2,1,1,1,2,1,1,...,3,1,3,3,2,2,1,2,1,4
2,2,2,6,3,2,1,2,1,2,2,...,2,3,1,1,1,1,1,1,1,2
3,3,2,3,9,1,2,3,2,1,4,...,2,4,0,1,1,3,1,2,1,2
4,1,1,2,1,5,3,1,3,4,1,...,1,0,2,1,1,2,2,1,2,1


In [52]:
dot_prod_shows_df = pd.read_csv('data/similar-shows.csv')

In [63]:
dot_prod_shows_df = dot_prod_shows_df.drop(['Unnamed: 0'], axis=1)

In [64]:
dot_prod_shows_df.head()

,0,1,2,3,4,5,6,7,8,9,...,16358,16359,16360,16361,16362,16363,16364,16365,16366,16367
0,6,4,2,3,1,2,1,2,2,1,...,4,2,1,3,2,3,2,3,1,4
1,4,6,2,2,1,1,1,2,1,1,...,3,1,3,3,2,2,1,2,1,4
2,2,2,6,3,2,1,2,1,2,2,...,2,3,1,1,1,1,1,1,1,2
3,3,2,3,9,1,2,3,2,1,4,...,2,4,0,1,1,3,1,2,1,2
4,1,1,2,1,5,3,1,3,4,1,...,1,0,2,1,1,2,2,1,2,1


In [58]:
dot_prod_shows_df.iloc[772]

Unnamed: 0    772
0               2
1               2
2               3
3               4
             ... 
16363           4
16364           1
16365           2
16366           2
16367           1
Name: 772, Length: 16369, dtype: int64

In [53]:
show_idx = np.where(animes_clean_df['uid'] == 11061)[0][0]
show_idx

772

In [66]:
similar_idxs = np.where(dot_prod_shows_df.iloc[show_idx] > np.max(dot_prod_shows_df.iloc[show_idx])-2)[0]
similar_idxs

array([   21,    64,   146,   195,   209,   241,   284,   317,   327,
         405,   420,   422,   423,   456,   459,   483,   503,   523,
         531,   536,   578,   619,   652,   673,   681,   689,   707,
         770,   772,   810,   811,   833,   850,   891,   902,   923,
         939,   950,  1041,  1042,  1064,  1258,  1299,  3123,  3196,
        3241,  3378,  3503,  3785,  3874,  4027,  4133,  4214,  4278,
        4479,  4634,  4775,  5036,  5071,  5162,  5178,  5321,  5380,
        5502,  5581,  5591,  5639,  5678,  5722,  5729,  5733,  5799,
        5809,  5831,  5921,  5938,  5997,  6057,  6087,  6107,  6287,
        6417,  6697,  6970,  7267,  7269,  7896,  7919,  8614,  8938,
        9450, 10395, 10425, 10534, 10535, 10708, 10931, 10948, 11307,
       11799, 13080, 13476, 13477, 14072, 14233, 14505, 14733, 14926,
       14929, 15119, 15172, 15291, 15310, 15347, 15382, 15511, 15534,
       15559, 15598, 15663, 15738, 15756, 15784, 15856, 15857, 15860,
       15866, 15909,

In [ ]:
similar_shows = animes_df.iloc[similar_idxs, ]
similar_shows

In [8]:
dot_prod_movies.to_csv('./data/similar-shows.csv')

In [49]:
np.where(dot_prod_movies[772] > np.max(dot_prod_movies[772])-2)[0]

array([   21,    64,   146,   195,   209,   241,   284,   317,   327,
         405,   420,   422,   423,   456,   459,   483,   503,   523,
         531,   536,   578,   619,   652,   673,   681,   689,   707,
         770,   772,   810,   811,   833,   850,   891,   902,   923,
         939,   950,  1041,  1042,  1064,  1258,  1299,  3123,  3196,
        3241,  3378,  3503,  3785,  3874,  4027,  4133,  4214,  4278,
        4479,  4634,  4775,  5036,  5071,  5162,  5178,  5321,  5380,
        5502,  5581,  5591,  5639,  5678,  5722,  5729,  5733,  5799,
        5809,  5831,  5921,  5938,  5997,  6057,  6087,  6107,  6287,
        6417,  6697,  6970,  7267,  7269,  7896,  7919,  8614,  8938,
        9450, 10395, 10425, 10534, 10535, 10708, 10931, 10948, 11307,
       11799, 13080, 13476, 13477, 14072, 14233, 14505, 14733, 14926,
       14929, 15119, 15172, 15291, 15310, 15347, 15382, 15511, 15534,
       15559, 15598, 15663, 15738, 15756, 15784, 15856, 15857, 15860,
       15866, 15909,

In [98]:
def get_show_by_id(anime_id, df):
    '''
    INPUT:
    show_name - a string of the anime title
    df - the clean animes df in the data directory

    OUTPUT:
    show.iloc[0] - the entire row in the df with all the show info
    '''
    show = df[df['uid'] == int(anime_id)]['title']
    return show

In [102]:
show_name = get_show_by_id('11061', animes_clean_df)
show_name.values[0]

'Hunter x Hunter (2011)'

In [17]:
a = '|938|4820|49030'
x = a.split('|')
x.pop(0)
x

['938', '4820', '49030']

In [71]:
show_idx = np.where(animes_df['uid'] == 2904)[0][0]
similar_idxs = np.where(dot_prod_movies[show_idx] == np.max(dot_prod_movies[show_idx])-2)[0]
    
similar_shows = animes_df.iloc[similar_idxs, ]
similar_shows.sort_values(by=['score'], ascending=False)

,uid,title,synopsis,episodes,members,popularity,ranked,score,img_url,link,...,Thriller,Vampire,Yaoi,Yuri,Pre 1970s,1970s,1980s,1990s,2000s,2010s
718,4565,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,"Humans have enjoyed their lavish, peaceful, an...",1.0,142624,776,74.0,8.58,https://cdn.myanimelist.net/images/anime/12/19...,https://myanimelist.net/anime/4565/Tengen_Topp...,...,0,0,0,0,0,0,0,0,1,0
774,801,Koukaku Kidoutai: Stand Alone Complex 2nd GIG,"Following the closure of the ""Laughing Man"" ca...",26.0,173629,626,88.0,8.52,https://cdn.myanimelist.net/images/anime/11/51...,https://myanimelist.net/anime/801/Koukaku_Kido...,...,0,0,0,0,0,0,0,0,1,0
663,467,Koukaku Kidoutai: Stand Alone Complex,"In the not so distant future, mankind has adva...",26.0,291249,331,129.0,8.44,https://cdn.myanimelist.net/images/anime/11/50...,https://myanimelist.net/anime/467/Koukaku_Kido...,...,0,0,0,0,0,0,0,0,1,0
474,6336,Mobile Suit Gundam Unicorn,"In the year Universal Century 0096, three year...",7.0,65690,1569,328.0,8.19,https://cdn.myanimelist.net/images/anime/12/50...,https://myanimelist.net/anime/6336/Mobile_Suit...,...,0,0,0,0,0,0,0,0,0,1
416,34440,Code Geass: Hangyaku no Lelouch III - Oudou,Lelouch Lamperouge's dreams of destroying the ...,1.0,42319,2122,385.0,8.15,https://cdn.myanimelist.net/images/anime/1126/...,https://myanimelist.net/anime/34440/Code_Geass...,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3525,2453,Dennou Sentai Voogie's★Angel,A hundred years after Earth is invaded by alie...,3.0,1731,8265,9646.0,5.58,https://cdn.myanimelist.net/images/anime/8/751...,https://myanimelist.net/anime/2453/Dennou_Sent...,...,0,0,0,0,0,0,0,1,0,0
7247,3070,Wakusei Daikaiju Negadon,"A world of the future, madly over populated, s...",1.0,1074,9372,9704.0,5.57,https://cdn.myanimelist.net/images/anime/2/460...,https://myanimelist.net/anime/3070/Wakusei_Dai...,...,0,0,0,0,0,0,0,0,1,0
1513,38717,Mobile Suit Gundam: Hathaway's Flash 3,NaN,1.0,910,9799,NaN,NaN,https://cdn.myanimelist.net/images/anime/1214/...,https://myanimelist.net/anime/38717/Mobile_Sui...,...,0,0,0,0,0,0,0,0,0,0
1514,38716,Mobile Suit Gundam: Hathaway's Flash 2,NaN,1.0,950,9695,NaN,NaN,https://cdn.myanimelist.net/images/anime/1019/...,https://myanimelist.net/anime/38716/Mobile_Sui...,...,0,0,0,0,0,0,0,0,0,0


In [75]:
def find_similar_shows(anime_id):
    '''
    Finds similar shows based on what genres/decades they have in common
    
    INPUT:
    anime_id - int, id of anime show that appears in the animes_df
    
    OUTPUT:
    similar_shows - pandas dataframe of similar shows sorted by highest rated
    '''
    show_idx = np.where(animes_clean_df['uid'] == anime_id)[0][0]
    
    similar_idxs = np.where(dot_prod_shows_df.iloc[show_idx] > np.max(dot_prod_shows_df.iloc[show_idx])-2)[0]
    
    similar_shows = animes_df.iloc[similar_idxs, ]
    similar_shows = similar_shows[similar_shows['uid'] != anime_id]
    similar_shows.sort_values(by=['score'], ascending=False)
    
    
    return similar_shows

In [97]:
a = find_similar_shows(11061)
for name, img in zip(a['title'], a['img_url']):
    print(name, img)

Saint Seiya: Meiou Hades Meikai-hen https://cdn.myanimelist.net/images/anime/12/75732.jpg
Shingeki no Kyojin OVA https://cdn.myanimelist.net/images/anime/9/59221.jpg
Magi: Sinbad no Bouken https://cdn.myanimelist.net/images/anime/13/60471.jpg
Fairy Tail (2014) https://cdn.myanimelist.net/images/anime/3/60551.jpg
Kekkai Sensen & Beyond https://cdn.myanimelist.net/images/anime/3/88282.jpg
One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken https://cdn.myanimelist.net/images/anime/10/87473.jpg
Magi: Sinbad no Bouken (TV) https://cdn.myanimelist.net/images/anime/10/78783.jpg
One Piece Film: Gold https://cdn.myanimelist.net/images/anime/12/81081.jpg
One Piece Film: Strong World Episode 0 https://cdn.myanimelist.net/images/anime/2/24152.jpg
Nanatsu no Taizai https://cdn.myanimelist.net/images/anime/8/65409.jpg
Saint Seiya: The Lost Canvas - Meiou Shinwa 2 https://cdn.myanimelist.net/images/anime/12/29597.jpg
One Piece: Episode of Nami - Koukaishi no Namida to Nakama no Ki